In [1]:
import numpy as np
from pathlib import Path
from tqdm import tqdm
import pandas as pd
from tbparse import SummaryReader

pd.set_option('display.max_columns', None)

models = {
    "Plen (sigma=10)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/thresh_10/ckpt/cf.txt",
    "Plen (sigma=30)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/thresh_30/ckpt/cf.txt",
    "Plen (sigma=50)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/thresh_50/ckpt/cf.txt",
    "Plen (sigma=70)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/thresh_70/ckpt/cf.txt",
    "Plen (sigma=90)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/thresh_90/ckpt/cf.txt",
    "Plen (sigma=100)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/thresh_100/ckpt/cf.txt",
    # "Plen (multi)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/thresh_multi/ckpt/cf.txt",
    # "NeRF (sigma=200)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/thresh_200/ckpt/cf.txt",
    # "NeRF (mean 0.1)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/depth_mean_0.1/ckpt/cf.txt",
    # "NeRF (mean 0.9)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/depth_mean_0.9/ckpt/cf.txt",
    # "NeRF (mode 0.1)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/depth_mode_0.1/ckpt/cf.txt",
    # "NeRF (mode 0.9)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/depth_mode_0.9/ckpt/cf.txt",
    # "NeRF (test mean 0.1)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/depth_spiral_mean_0.1/ckpt/cf.txt",
    # "NeRF (test mode 0.1)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/depth_spiral_mode_0.1/ckpt/cf.txt",

    "Mip360 (sigma=10)": "/rds/project/rds-qxpdOeYWi78/multinerf/results/blender/{}/pts/lv_10_pts/cf.txt",
    "Mip360 (sigma=30)": "/rds/project/rds-qxpdOeYWi78/multinerf/results/blender/{}/pts/lv_30_pts/cf.txt",
    "Mip360 (sigma=50)": "/rds/project/rds-qxpdOeYWi78/multinerf/results/blender/{}/pts/lv_50_pts/cf.txt",
    "Mip360 (sigma=70)": "/rds/project/rds-qxpdOeYWi78/multinerf/results/blender/{}/pts/lv_70_pts/cf.txt",
    "Mip360 (sigma=90)": "/rds/project/rds-qxpdOeYWi78/multinerf/results/blender/{}/pts/lv_90_pts/cf.txt",
    "Mip360 (sigma=100)": "/rds/project/rds-qxpdOeYWi78/multinerf/results/blender/{}/pts/lv_100_pts/cf.txt",
    # "MipNeRF360 (sigma=200)": "/rds/project/rds-qxpdOeYWi78/multinerf/results/blender/{}/pts/lv_200_pts/cf.txt",

    "NeuS White" : "/rds/project/rds-qxpdOeYWi78/NeuS/exp/{}/womask_white/fine_mesh/cf.txt",
    "NeuS" : "/rds/project/rds-qxpdOeYWi78/NeuS/exp/{}/womask/fine_mesh/cf.txt",
    # "NeuS 0.5" : "/rds/project/rds-qxpdOeYWi78/NeuS/exp/{}/womask/fine_mesh/max_dist05/cf.txt",
    "HFS White" : "/rds/project/rds-qxpdOeYWi78/HFS/exps_1.5/{}/womask_white_no_bg/fine_mesh/cf.txt",
    "HFS" : "/rds/project/rds-qxpdOeYWi78/HFS/exps_1.5/{}/womask_hfs/fine_mesh/cf.txt",
    # "HFS 0.5" : "/rds/project/rds-qxpdOeYWi78/HFS/exps_1.5/{}/womask_hfs/fine_mesh/max_dist05/cf.txt",


    # "Ours": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/good_trunc/solid_less_trunc/ckpt_eval_surf_single/ckpt/cf.txt",
    "Ours": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/good_trunc/solid_less_trunc/ckpt_eval_surf_masked/ckpt/cf.txt",
    "Ours_new_trans": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/norm_l12/norm_l12_decay_9/ckpt_eval_surf_masked/ckpt/cf.txt",
    # "Ours_new_trans (single mask)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/norm_l12/norm_l12_decay_9/ckpt_eval_surf_single_masked/ckpt/cf.txt",
    # "Ours (new_trans masked)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/norm_l12/norm_l12_decay_9/ckpt_eval_surf_masked/ckpt/cf.txt",


    

    "NeRF (depth 0.1)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/depth_spiral_mode_0.1/remeshed/cf.txt",
    "Mip360 (depth 0.9)": "/rds/project/rds-qxpdOeYWi78/multinerf/results/blender/{}/pts/distance_mode_09_pts/cf.txt",
    "Mip360 (depth 0.1)": "/rds/project/rds-qxpdOeYWi78/multinerf/results/blender/{}/pts/distance_mode_pts/cf.txt",
    
    

    # "Ours (new_trans)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/norm_l12/norm_l12_decay_9/ckpt_eval_surf_single/ckpt/cf.txt",

}


def read_entry(p, scale=1e-3):
    entry = {
        "name": "",
        # "Mean d2s": None,
        # "Mean s2d": None,
        "Avg cf": None,
        # "Accuracy": None,
        # "Completeness": None,
        # "F1": None,
        # "psnr": None,
    }
    if Path(p).exists():
        with Path(p).open('r') as f:
            lines = f.readlines()

            for k in entry.keys():
                for l in lines:
                    if l.startswith(k):
                        value = l.split(':')[-1].strip()
                        if value != 'None':
                            entry[k] = float(value) 
                            if k == 'Avg cf': entry[k]/= scale
    
    return entry

def get_scene_comp(scene_name, scale=1e-3):
    df = pd.DataFrame()
    for name in models.keys():
        e = read_entry(models[name].format(scene_name), scale)
        e['name'] = name

        df = pd.concat([df, pd.DataFrame(e, index=[len(df)])])
    return df

replace_dict = {
    'ship\_re':'ship',
    'lego\_re':'lego',
    'mic\_re':'mic',
    'hotdog\_re':'hotdog',
    'chair\_re':'chair',
    '(sigma=10)':'($\sigma=10$)',
    '(sigma=50)':'($\sigma=50$)',
    '(sigma=100)':'($\sigma=100$)',
    'test mode':'depth',
    'Ours (clamp\_0149\_no\_ldist\_no\_in\_sparse/surf\_sin...':'Ours',
    'Ours (clamp\_0149\_no\_ldist\_no\_in\_sparse/surf\_mas...':'Ours (trimmed)',
    'Ours (solid\_no\_fs/surf\_single)': 'Ours',
    'Ours (solid\_no\_fs/surf\_masked)': 'Ours (trimmed)',
    'Ours (trans\_no\_fs\_low\_sparse\_2/surf\_single)': 'Ours',
    'Ours (trans\_no\_fs\_low\_sparse\_2/surf\_masked)': 'Ours (trimmed)',

}

def highlight_second_min(s):
    styles = []
    min_value = s.min()
    second_min_value = s[s > min_value].min()
    for v in s:
        if v == second_min_value:
            styles.append('background-color: violet')
        else:
            styles.append('')
    return styles

def highlight_third_min(s):
    styles = []
    min_value = s.min()
    second_min_value = s[s > min_value].min()
    third_min_value = s[s > second_min_value].min()
    for v in s:
        if v == third_min_value:
            styles.append('background-color: thistle')
        else:
            styles.append('')
    return styles

def highlight_min_values(s):
    styles = []
    sorted_s = s.sort_values()
    min_value = sorted_s.iloc[0]
    second_min_value = sorted_s.iloc[1]
    third_min_value = sorted_s.iloc[2]
    for v in s:
        if v == min_value:
            styles.append('background-color: magenta')
        elif v == second_min_value:
            styles.append('background-color: violet')
        # elif v == third_min_value:
        #     styles.append('background-color: thistle')
        else:
            styles.append('')
    return styles

def mark_best_values(s, second=True):
    modified_values = []
    # max_value = s.max()
    if 'psnr' in s.name or 'ssim' in s.name:
        sorted_s = s.sort_values(ascending=False)
    else:
        sorted_s = s.sort_values()

    best_value = sorted_s.iloc[0]
    second_best_value = sorted_s.iloc[1]
    for v in s:
        if v == best_value:
            modified_values.append(f'\\first{{{v}}}')
        elif v == second_best_value:
            if second:
                modified_values.append(f'\\second{{{v}}}')
            else:
                modified_values.append(v)
        else:
            modified_values.append(v)
    return modified_values

# DTU

In [39]:
scene_names = [
    "dtu_scan24", 
    "dtu_scan37",
    "dtu_scan40",
    "dtu_scan55",
    "dtu_scan63", 
    "dtu_scan65",
    "dtu_scan83", 
    "dtu_scan69",
    "dtu_scan97",
    "dtu_scan105", 
    "dtu_scan106",
    "dtu_scan110",
    "dtu_scan114",
    "dtu_scan118",
    "dtu_scan122",
]



models = {
    "NeRF (sigma=10)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/thresh_10/ckpt/cf.txt",
    "NeRF (sigma=50)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/thresh_50/ckpt/cf.txt",
    "NeRF (sigma=100)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/thresh_100/ckpt/cf.txt",

    # "NeRF black (sigma=10)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn_black/ckpt_eval_cuvol/thresh_10/ckpt/cf.txt",
    # "NeRF black (sigma=50)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn_black/ckpt_eval_cuvol/thresh_50/ckpt/cf.txt",
    # "NeRF black (sigma=100)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn_black/ckpt_eval_cuvol/thresh_100/ckpt/cf.txt",

    # "Ours sparse": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/dtu_new_exp/sparse/ckpt_eval_surf_single/ckpt/cf.txt",
    # "Ours sparse (trimmed)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/dtu_new_exp/sparse/ckpt_eval_surf_masked/ckpt/cf.txt",
    
    # "Ours less_sparse": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/dtu_new_exp/less_sparse/ckpt_eval_surf_single/ckpt/cf.txt",
    # "Ours less_sparse (trimmed)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/dtu_new_exp/less_sparse/ckpt_eval_surf_masked/ckpt/cf.txt",
    
    "Ours less_trunc": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/dtu_new_exp/less_trunc/ckpt_eval_surf_single/ckpt/cf.txt",
    "Ours less_trunc (trimmed)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/dtu_new_exp/less_trunc/ckpt_eval_surf_masked/ckpt/cf.txt",
    
    # "Ours less_trunc_black": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/dtu_new_exp/less_trunc_black/ckpt_eval_surf_single/ckpt/cf.txt",
    # "Ours less_trunc_black (trimmed)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/dtu_new_exp/less_trunc_black/ckpt_eval_surf_masked/ckpt/cf.txt",
    
    # "Ours less_trunc_less_sparse": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/dtu_new_exp/less_trunc_less_sparse/ckpt_eval_surf_single/ckpt/cf.txt",
    
    # "Ours less_trunc_2": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/dtu_new_exp/less_trunc_2/ckpt_eval_surf_single/ckpt/cf.txt",
    # "Ours less_trunc_2 (trimmed)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/dtu_new_exp/less_trunc_2/ckpt_eval_surf_masked/ckpt/cf.txt",

    # "NeRF (depth 0.1)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/depth_spiral_mode_0.1/remeshed/cf.txt",
}


df_all = None

digit = 2

for i, scene in enumerate(scene_names):
    df = get_scene_comp(scene, 1)
    df = df.round(digit)
    # df.loc[-1] = ["scene name", scene, scene]
    df = df.rename(columns={'Avg cf':scene})
    # df.loc[-1] = ["scene name", scene]
    # df.index = df.index + 1  # shifting index
    # df.sort_index(inplace=True)

    if i > 0:
        df = df.drop('name', axis=1)
    df_all = pd.concat([df_all, df], axis=1)

    

# neus = ['neus', 1.37, 1.21, 0.73, 0.40, 1.20, 0.70, 0.72, 1.01, 1.116, 0.82, 0.66, 1.69, 0.39, 0.49, 0.51]
# df_all.loc['NeuS'] = ['neus', 1.37, 1.21, 0.73, 0.40, 1.20, 0.70, 0.72, 1.01, 1.116, 0.82, 0.66, 1.69, 0.39, 0.49, 0.51]

# df_all['mean cf'] = df_all['Avg cf'][1:].mean(axis=1).round(digit)
df_all['avg'] = df_all[scene_names].mean(axis=1).round(digit)
# df_all['mean F1'] = df_all['F1'][1:].mean(axis=1).round(digit)


# df_all.style.highlight_min(color = 'purple', axis = 0).apply(highlight_second_min).apply(highlight_third_min)
df_all.style.apply(highlight_min_values)

,name,dtu_scan24,dtu_scan37,dtu_scan40,dtu_scan55,dtu_scan63,dtu_scan65,dtu_scan83,dtu_scan69,dtu_scan97,dtu_scan105,dtu_scan106,dtu_scan110,dtu_scan114,dtu_scan118,dtu_scan122,avg
0,NeRF (sigma=10),1.990000,1.900000,1.860000,1.710000,1.860000,1.960000,2.220000,2.040000,2.430000,1.820000,1.850000,1.960000,1.670000,1.990000,1.710000,1.930000
1,NeRF (sigma=50),1.370000,1.460000,1.430000,1.220000,1.660000,1.850000,2.030000,1.600000,1.980000,1.260000,1.490000,1.750000,1.450000,1.280000,1.210000,1.540000
2,NeRF (sigma=100),1.510000,1.340000,1.570000,0.880000,2.990000,2.000000,2.840000,2.220000,2.800000,2.080000,1.720000,2.430000,1.440000,1.910000,1.650000,1.960000
3,Ours less_trunc,1.470000,1.660000,1.590000,1.340000,1.180000,2.260000,1.530000,2.300000,3.360000,1.310000,1.830000,1.590000,1.310000,2.220000,1.450000,1.760000
4,Ours less_trunc (trimmed),1.040000,1.360000,1.300000,1.100000,0.890000,2.150000,1.290000,1.900000,2.840000,0.990000,1.710000,1.330000,1.090000,1.880000,1.240000,1.470000


In [62]:
scene_names = [
    "dtu_scan24", 
    "dtu_scan37",
    "dtu_scan40",
    "dtu_scan55",
    "dtu_scan63", 
    "dtu_scan65",
    "dtu_scan83", 
    "dtu_scan69",
    "dtu_scan97",
    "dtu_scan105", 
    "dtu_scan106",
    "dtu_scan110",
    "dtu_scan114",
    "dtu_scan118",
    "dtu_scan122",
]



models = {
    "NeRF (sigma=10)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/thresh_10/ckpt/cf.txt",
    "NeRF (sigma=50)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/thresh_50/ckpt/cf.txt",
    "NeRF (sigma=100)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/thresh_100/ckpt/cf.txt",

    # "NeRF black (sigma=10)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn_black/ckpt_eval_cuvol/thresh_10/ckpt/cf.txt",
    # "NeRF black (sigma=50)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn_black/ckpt_eval_cuvol/thresh_50/ckpt/cf.txt",
    # "NeRF black (sigma=100)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn_black/ckpt_eval_cuvol/thresh_100/ckpt/cf.txt",

    # "Ours sparse": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/dtu_new_exp/sparse/ckpt_eval_surf_single/ckpt/cf.txt",
    # "Ours sparse (trimmed)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/dtu_new_exp/sparse/ckpt_eval_surf_masked/ckpt/cf.txt",
    
    # "Ours less_sparse": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/dtu_new_exp/less_sparse/ckpt_eval_surf_single/ckpt/cf.txt",
    # "Ours less_sparse (trimmed)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/dtu_new_exp/less_sparse/ckpt_eval_surf_masked/ckpt/cf.txt",
    
#     "NeRF (depth 0.1)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/depth_spiral_mode_0.1/remeshed/cf.txt",
#     "Ours less_trunc": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/dtu_new_exp/less_trunc/ckpt_eval_surf_single/ckpt/cf.txt",
    "Ours less_trunc all surf": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/dtu_new_exp/less_trunc/ckpt_eval_surf_mlv/ckpt/cf.txt",
#     "Ours less_trunc (trimmed)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/dtu_new_exp/less_trunc/ckpt_eval_surf_masked/ckpt/cf.txt",
    
#     "Ours less_trunc_norm all surf": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/dtu_new_exp/less_trunc_norm_59/ckpt_eval_surf_mlv/ckpt/cf.txt",
    
    # "Ours less_trunc_black": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/dtu_new_exp/less_trunc_black/ckpt_eval_surf_single/ckpt/cf.txt",
    # "Ours less_trunc_black (trimmed)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/dtu_new_exp/less_trunc_black/ckpt_eval_surf_masked/ckpt/cf.txt",
    
    # "Ours less_trunc_less_sparse": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/dtu_new_exp/less_trunc_less_sparse/ckpt_eval_surf_single/ckpt/cf.txt",
    
    # "Ours less_trunc_2": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/dtu_new_exp/less_trunc_2/ckpt_eval_surf_single/ckpt/cf.txt",
    # "Ours less_trunc_2 (trimmed)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/dtu_new_exp/less_trunc_2/ckpt_eval_surf_masked/ckpt/cf.txt",

}


df_all = None

digit = 2

for i, scene in enumerate(scene_names):
    df = get_scene_comp(scene, 1)
    df = df.round(digit)
    # df.loc[-1] = ["scene name", scene, scene]
    df = df.rename(columns={'Avg cf':scene})
    # df.loc[-1] = ["scene name", scene]
    # df.index = df.index + 1  # shifting index
    # df.sort_index(inplace=True)

    if i > 0:
        df = df.drop('name', axis=1)
    df_all = pd.concat([df_all, df], axis=1)

    

# df_all.loc['NeuS'] = ['neus nomask', 1.37, 1.21, 0.73, 0.40, 1.20, 0.70, 0.72, 1.01, 1.116, 0.82, 0.66, 1.69, 0.39, 0.49, 0.51]
df_all.loc['NeuS m'] = ['neus mask', 0.83, 0.98, 0.56, 0.37, 1.13, 0.59, 0.60, 1.45, 0.95, 0.78, 0.52, 1.43, 0.36, 0.45, 0.45]

# df_all.loc['HFS'] = ['HFS', 0.76, 1.32, 0.70, 0.39, 1.06, 0.63, 0.63, 1.15, 1.12, 0.80, 0.52, 1.22, 0.33, 0.49, 0.50]

scenes = [
#     "dtu_scan24", 
    "dtu_scan37",
    "dtu_scan40",
#     "dtu_scan55",
    "dtu_scan63", 
#     "dtu_scan65",
#     "dtu_scan83", 
    "dtu_scan69",
#     "dtu_scan97",
#     "dtu_scan105", 
#     "dtu_scan106",
    "dtu_scan110",
#     "dtu_scan114",
#     "dtu_scan118",
#     "dtu_scan122",
]

df_all = df_all[['name', *scenes]]

# df_all['mean cf'] = df_all['Avg cf'][1:].mean(axis=1).round(digit)
# df_all['avg'] = df_all[scenes].mean(axis=1).round(digit)
# df_all['mean F1'] = df_all['F1'][1:].mean(axis=1).round(digit)

df_all.style.apply(highlight_min_values)

,name,dtu_scan37,dtu_scan40,dtu_scan63,dtu_scan69,dtu_scan110
0,NeRF (sigma=10),1.900000,1.860000,1.860000,2.040000,1.960000
1,NeRF (sigma=50),1.460000,1.430000,1.660000,1.600000,1.750000
2,NeRF (sigma=100),1.340000,1.570000,2.990000,2.220000,2.430000
3,Ours less_trunc all surf,1.340000,1.360000,0.990000,1.910000,1.370000
NeuS m,neus mask,0.980000,0.560000,1.130000,1.450000,1.430000


In [64]:
print(df_all.to_latex(index=False))

\begin{tabular}{lrrrrr}
\toprule
                    name &  dtu\_scan37 &  dtu\_scan40 &  dtu\_scan63 &  dtu\_scan69 &  dtu\_scan110 \\
\midrule
         NeRF (sigma=10) &        1.90 &        1.86 &        1.86 &        2.04 &         1.96 \\
         NeRF (sigma=50) &        1.46 &        1.43 &        1.66 &        1.60 &         1.75 \\
        NeRF (sigma=100) &        1.34 &        1.57 &        2.99 &        2.22 &         2.43 \\
Ours less\_trunc all surf &        1.34 &        1.36 &        0.99 &        1.91 &         1.37 \\
               neus mask &        0.98 &        0.56 &        1.13 &        1.45 &         1.43 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_29719/1159970361.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df_all.to_latex(index=False))


In [ ]:

\begin{tabular}{lrrrrr}
\toprule
                    name &  dtu\_scan37 &  dtu\_scan40 &  dtu\_scan63 &  dtu\_scan69 &  dtu\_scan110 \\
\midrule
         NeRF (sigma=10) &        1.90 &        1.86 &        1.86 &        2.04 &         1.96 \\
         NeRF (sigma=50) &        1.46 &        1.43 &        1.66 &        1.60 &         1.75 \\
        NeRF (sigma=100) &        1.34 &        1.57 &        2.99 &        2.22 &         2.43 \\
Ours                     &        1.34 &        1.36 &        0.99 &        1.91 &         1.37 \\
               neus mask &        0.98 &        0.56 &        1.13 &        1.45 &         1.43 \\
\bottomrule
\end{tabular}

In [11]:
df_all.loc['NeuS'] = ['neus', 1.37, 1.21, 0.73, 0.40, 1.20, 0.70, 0.72, 1.01, 1.116, 0.82, 0.66, 1.69, 0.39, 0.49, 0.51, 1]
df_all

,name,dtu_scan24,dtu_scan37,dtu_scan40,dtu_scan55,dtu_scan63,dtu_scan65,dtu_scan83,dtu_scan69,dtu_scan97,dtu_scan105,dtu_scan106,dtu_scan110,dtu_scan114,dtu_scan118,dtu_scan122,avg
0,NeRF (sigma=10),1.990,1.900,1.863,1.708,1.855,1.956,2.225,2.036,2.430,1.817,1.852,1.956,1.668,1.986,1.712,1.930
1,NeRF (sigma=50),1.375,1.460,1.430,1.218,1.663,1.848,2.026,1.600,1.978,1.263,1.485,1.745,1.452,1.280,1.211,1.536
2,NeRF (sigma=100),1.512,1.343,1.571,0.881,2.990,2.003,2.841,2.224,2.796,2.080,1.719,2.429,1.442,1.909,1.652,1.959
3,Ours less_sparse,1.690,1.842,1.562,1.212,1.301,2.192,1.695,2.550,3.862,1.595,1.785,1.654,1.452,2.470,1.618,1.899
4,Ours less_trunc,1.468,1.659,1.589,1.338,1.178,2.264,1.535,2.296,3.362,1.313,1.834,1.593,1.311,2.224,1.447,1.761
5,Ours less_trunc (trimmed),1.035,1.355,1.298,1.100,0.889,2.154,1.293,1.904,2.843,0.987,1.711,1.334,1.093,1.878,1.244,1.475
6,NeRF (depth 0.1),1.215,1.397,1.232,0.847,1.299,1.727,1.697,1.403,2.009,1.056,1.316,1.436,1.207,1.439,1.079,1.357
NeuS,neus,1.370,1.210,0.730,0.400,1.200,0.700,0.720,1.010,1.116,0.820,0.660,1.690,0.390,0.490,0.510,1.000


# Abla

In [11]:

models = {
    'no tv': "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/abla_less_trunc/no_tv/ckpt_eval_surf_masked/ckpt/cf.txt",
    'notvnorm': "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/abla_less_trunc/no_tv_no_norm/ckpt_eval_surf_masked/ckpt/cf.txt",
    'no trunc': "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/abla_less_trunc/no_trunc/ckpt_eval_surf_masked/ckpt/cf.txt",
    'no conv': "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/abla_less_trunc/no_conv/ckpt_eval_surf_masked/ckpt/cf.txt",
    'single lv': "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/abla_less_trunc/single_lv/ckpt_eval_surf_masked/ckpt/cf.txt",
    # 'no reg': "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/abla_less_trunc/no_reg/ckpt_eval_surf_masked/ckpt/cf.txt",
    "Ours": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/good_trunc/solid_less_trunc/ckpt_eval_surf_masked/ckpt/cf.txt",
}

# models = {
#     'no tv': "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/abla_solid_conv_lv/no_tv/ckpt_eval_surf_masked/ckpt/cf.txt",
#     'notvnorm': "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/abla_solid_conv_lv/no_tv_no_norm/ckpt_eval_surf_masked/ckpt/cf.txt",
#     'no trunc': "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/abla_solid_conv_lv/no_trunc/ckpt_eval_surf_masked/ckpt/cf.txt",
#     'no conv': "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/abla_solid_conv_lv/no_conv/ckpt_eval_surf_masked/ckpt/cf.txt",
#     'single lv': "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/abla_solid_conv_lv/single_lv/ckpt_eval_surf_masked/ckpt/cf.txt",
#     "Ours": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/good_trunc/solid_less_trunc_converge_lv/ckpt_eval_surf_masked/ckpt/cf.txt",
# }

scene_names = [
    "ship_re", 
    "ficus",
    
    # "lyre",
    # "stair",
    # "bee",
    # "fence",
    # "scale",
    # "seat",
    # "well",
    # "slide",
]

df_all = None

digit = 3

for i, scene in enumerate(scene_names):
    df = get_scene_comp(scene, 1e-2)
    df = df.round(digit)
    # df.loc[-1] = ["scene name", scene, scene]
    df = df.rename(columns={'Avg cf':scene})
    # df.loc[-1] = ["scene name", scene]
    # df.index = df.index + 1  # shifting index
    # df.sort_index(inplace=True)

    if i > 0:
        df = df.drop('name', axis=1)
    df_all = pd.concat([df_all, df], axis=1)

    

# df_all['mean cf'] = df_all['Avg cf'][1:].mean(axis=1).round(digit)
# df_all['avg'] = df_all[scene_names].mean(axis=1).round(digit)
# df_all['mean F1'] = df_all['F1'][1:].mean(axis=1).round(digit)
df_all


,name,ship_re,ficus
0,no tv,0.317,0.279
1,notvnorm,0.373,0.333
2,no trunc,0.522,0.837
3,no conv,0.287,0.266
4,single lv,0.624,0.583
5,Ours,0.277,0.240


In [12]:
# print(df_all.iloc[[0,1,2,3,4,5]].transpose().to_latex())
print()

df_latex = df_all.iloc[[0,1,2,3,4,5]].to_latex(index=False)

df_latex = df_latex.replace('0.', '.')

print(df_latex)


\begin{tabular}{lrr}
\toprule
     name &  ship\_re &  ficus \\
\midrule
    no tv &    .317 &  .279 \\
 notvnorm &    .373 &  .333 \\
 no trunc &    .522 &  .837 \\
  no conv &    .287 &  .266 \\
single lv &    .624 &  .583 \\
     Ours &    .277 &  .240 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_201552/744298599.py:4: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  df_latex = df_all.iloc[[0,1,2,3,4,5]].to_latex(index=False)


In [7]:
models = {
    
    'no tv': "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/abla_new_trans/no_tv/ckpt_eval_surf_masked/ckpt/cf.txt",
    
    'notvnorm': "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/abla_new_trans/no_tv_no_norm/ckpt_eval_surf_masked/ckpt/cf.txt",
        
    'no trunc': "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/abla_new_trans/no_trunc/ckpt_eval_surf_masked/ckpt/cf.txt",
    
    'no conv': "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/abla_new_trans/no_conv/ckpt_eval_surf_masked/ckpt/cf.txt",
    
    'single lv': "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/abla_new_trans/single_lv/ckpt_eval_surf_masked/ckpt/cf.txt",
    
    "Ours": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/norm_l12/norm_l12_decay_9/ckpt_eval_surf_masked/ckpt/cf.txt",
    "norm_l12_decay_9_low_tv_2": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/norm_l12/norm_l12_decay_9_low_tv_2/ckpt_eval_surf_masked/ckpt/cf.txt",
    # "norm_l12_decay_9_low_tv_3": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/norm_l12/norm_l12_decay_9_low_tv_3/ckpt_eval_surf_masked/ckpt/cf.txt",
    

}

# models = {
#     'no tv': "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/abla_new_trans_conv_lv/no_tv/ckpt_eval_surf_masked/ckpt/cf.txt",
#     'notvnorm': "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/abla_new_trans_conv_lv/no_tv_no_norm/ckpt_eval_surf_masked/ckpt/cf.txt",
#     'no trunc': "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/abla_new_trans_conv_lv/no_trunc/ckpt_eval_surf_masked/ckpt/cf.txt",
#     'no conv': "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/abla_new_trans_conv_lv/no_conv/ckpt_eval_surf_masked/ckpt/cf.txt",
#     'single lv': "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/abla_new_trans_conv_lv/single_lv/ckpt_eval_surf_masked/ckpt/cf.txt",
#     "Ours": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/norm_l12/norm_l12_decay_9_converge_lv_2/ckpt_eval_surf_masked/ckpt/cf.txt",
#     "norm_l12_decay_9_low_tv_2": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/norm_l12/norm_l12_decay_9_low_tv_2/ckpt_eval_surf_masked/ckpt/cf.txt",
# }


## abla_new_trans_conv_lv_low_tv is actually abla_new_trans_low_tv
models = {
    'no tv': "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/abla_new_trans_low_tv/no_tv/ckpt_eval_surf_masked/ckpt/cf.txt",
    'notvnorm': "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/abla_new_trans_low_tv/no_tv_no_norm/ckpt_eval_surf_masked/ckpt/cf.txt",
    'no trunc': "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/abla_new_trans_low_tv/no_trunc/ckpt_eval_surf_masked/ckpt/cf.txt",
    'no conv': "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/abla_new_trans_low_tv/no_conv/ckpt_eval_surf_masked/ckpt/cf.txt",
    'single lv': "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/abla_new_trans_low_tv/single_lv/ckpt_eval_surf_masked/ckpt/cf.txt",
    # 'no_reg': "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/abla_new_trans_low_tv/no_reg/ckpt_eval_surf_masked/ckpt/cf.txt",
    # "norm_l12_decay_9_low_tv_2_conv_lv": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/norm_l12/norm_l12_decay_9_low_tv_2_conv_lv/ckpt_eval_surf_masked/ckpt/cf.txt",
    "Ours": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/norm_l12/norm_l12_decay_9_low_tv_2/ckpt_eval_surf_masked/ckpt/cf.txt",
}

scene_names = [
    # "case",
    "dinning_table",
    "monkey",
    # "coffe_table",
    # "kitchen_table",
    # "bottle_2",
    # "double_table",
    # "leaf_vase"
]

df_all = None

digit = 3

for i, scene in enumerate(scene_names):
    df = get_scene_comp(scene, 1e-2)
    df = df.round(digit)
    # df.loc[-1] = ["scene name", scene, scene]
    df = df.rename(columns={'Avg cf':scene})
    # df.loc[-1] = ["scene name", scene]
    # df.index = df.index + 1  # shifting index
    # df.sort_index(inplace=True)

    if i > 0:
        df = df.drop('name', axis=1)
    df_all = pd.concat([df_all, df], axis=1)

    

# df_all['mean cf'] = df_all['Avg cf'][1:].mean(axis=1).round(digit)
# df_all['avg'] = df_all[scene_names].mean(axis=1).round(digit)
# df_all['mean F1'] = df_all['F1'][1:].mean(axis=1).round(digit)
df_all

,name,dinning_table,monkey
0,no tv,0.373,0.776
1,notvnorm,0.404,0.970
2,no trunc,0.431,1.028
3,no conv,0.412,0.857
4,single lv,0.470,0.946
5,Ours,0.373,0.776


In [9]:
# print(df_all.iloc[[0,1,2,3,4,5]].transpose().to_latex())
print()

df_latex = df_all.iloc[[0,1,2,3,4,5]].to_latex(index=False)

df_latex = df_latex.replace('0.', '.')

print(df_latex)


\begin{tabular}{lrr}
\toprule
     name &  dinning\_table &  monkey \\
\midrule
    no tv &          .373 &   .776 \\
 notvnorm &          .404 &   .970 \\
 no trunc &          .431 &   1.028 \\
  no conv &          .412 &   .857 \\
single lv &          .470 &   .946 \\
     Ours &          .373 &   .776 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_201552/744298599.py:4: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  df_latex = df_all.iloc[[0,1,2,3,4,5]].to_latex(index=False)


In [ ]:
\toprule
     name &  ship\_re &  ficus & dinning\_table &  monkey \\
\midrule
    no tv &    .317 &  .279 & .373 &   .776 \\
 notvnorm &    .373 &  .333 & .404 &   .970 \\
 no trunc &    .522 &  .837 & .431 &   1.028 \\
  no conv &    .287 &  .266 & .412 &   .857 \\
single lv &    .624 &  .583 & .470 &   .946 \\
     Ours &    .277 &  .240 & .373 &   .776 \\
\bottomrule

# NeRF synthetic dataset (deprecated)

In [2]:
scene_names = [
    "ship_re", 
    "lego_re", 
    "ficus",
    "drums",
    "mic_re",
    "materials",
    "hotdog_re",
    "chair_re",
]


models = {
    # "Plen (sigma=10)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/thresh_10/ckpt/cf.txt",
    # "Plen (sigma=30)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/thresh_30/ckpt/cf.txt",
    # "Plen (sigma=50)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/thresh_50/ckpt/cf.txt",
    "Plen (sigma=70)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/thresh_70/ckpt/cf.txt",
    # "Plen (sigma=90)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/thresh_90/ckpt/cf.txt",
    # "Plen (sigma=100)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/thresh_100/ckpt/cf.txt",

    # "Mip360 (sigma=10)": "/rds/project/rds-qxpdOeYWi78/multinerf/results/blender/{}/pts/lv_10_pts/cf.txt",
    "Mip360 (sigma=30)": "/rds/project/rds-qxpdOeYWi78/multinerf/results/blender/{}/pts/lv_30_pts/cf.txt",
    # "Mip360 (sigma=50)": "/rds/project/rds-qxpdOeYWi78/multinerf/results/blender/{}/pts/lv_50_pts/cf.txt",
    # "Mip360 (sigma=70)": "/rds/project/rds-qxpdOeYWi78/multinerf/results/blender/{}/pts/lv_70_pts/cf.txt",
    # "Mip360 (sigma=90)": "/rds/project/rds-qxpdOeYWi78/multinerf/results/blender/{}/pts/lv_90_pts/cf.txt",
    # "Mip360 (sigma=100)": "/rds/project/rds-qxpdOeYWi78/multinerf/results/blender/{}/pts/lv_100_pts/cf.txt",
    # "MipNeRF360 (sigma=200)": "/rds/project/rds-qxpdOeYWi78/multinerf/results/blender/{}/pts/lv_200_pts/cf.txt",

    # "NeuS White" : "/rds/project/rds-qxpdOeYWi78/NeuS/exp/{}/womask_white/fine_mesh/cf.txt",
    "NeuS" : "/rds/project/rds-qxpdOeYWi78/NeuS/exp/{}/womask/fine_mesh/cf.txt",
    # "HFS White" : "/rds/project/rds-qxpdOeYWi78/HFS/exps_1.5/{}/womask_white_no_bg/fine_mesh/cf.txt",
    "HFS" : "/rds/project/rds-qxpdOeYWi78/HFS/exps_1.5/{}/womask_hfs/fine_mesh/cf.txt",
    
    "neuralangelo" : "/rds/project/rds-qxpdOeYWi78/neuralangelo/logs/syn/{}/cf.txt",


    # "Ours (Conv Lv)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/good_trunc/solid_less_trunc_converge_lv/ckpt_eval_surf_masked/ckpt/cf.txt",
    "Ours": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/good_trunc/solid_less_trunc/ckpt_eval_surf_masked/ckpt/cf.txt",
    # "Ours_new_trans": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/norm_l12/norm_l12_decay_9/ckpt_eval_surf_masked/ckpt/cf.txt",
    

    # "NeRF (depth 0.1)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/depth_spiral_mode_0.1/remeshed/cf.txt",
    # # "Mip360 (depth 0.9)": "/rds/project/rds-qxpdOeYWi78/multinerf/results/blender/{}/pts/distance_mode_09_pts/cf.txt",
    # "Mip360 (depth 0.1)": "/rds/project/rds-qxpdOeYWi78/multinerf/results/blender/{}/pts/distance_mode_pts/cf.txt",

}

df_all = None

digit = 3

for i, scene in enumerate(scene_names):
    df = get_scene_comp(scene, 1e-2)
    df = df.round(digit)
    # df.loc[-1] = ["scene name", scene, scene]
    df = df.rename(columns={'Avg cf':scene})
    # df.loc[-1] = ["scene name", scene]
    # df.index = df.index + 1  # shifting index
    # df.sort_index(inplace=True)

    if i > 0:
        df = df.drop('name', axis=1)
    df_all = pd.concat([df_all, df], axis=1)

    

# df_all['mean cf'] = df_all['Avg cf'][1:].mean(axis=1).round(digit)
df_all['avg'] = df_all[scene_names].mean(axis=1).round(digit)
# df_all['mean F1'] = df_all['F1'][1:].mean(axis=1).round(digit)
# df_all.style.highlight_min(color = 'grey', axis = 0)

df_all.style.apply(highlight_min_values)

,name,ship_re,lego_re,ficus,drums,mic_re,materials,hotdog_re,chair_re,avg
0,Plen (sigma=70),0.511000,0.560000,0.524000,1.216000,0.883000,0.907000,1.244000,1.003000,0.856000
1,Mip360 (sigma=30),1.122000,0.798000,2.163000,2.464000,1.470000,1.702000,1.482000,1.905000,1.638000
2,NeuS,0.552000,0.619000,1.667000,10.273000,0.251000,0.210000,0.602000,0.210000,1.798000
3,HFS,0.514000,0.463000,0.374000,1.820000,0.274000,0.255000,0.272000,0.428000,0.550000
4,neuralangelo,0.270000,nan,0.411000,nan,nan,nan,nan,nan,0.340000
5,Ours,0.277000,0.376000,0.240000,0.617000,0.288000,0.317000,0.702000,0.349000,0.396000


In [3]:
table = df_all.to_latex(index=False)



for k in replace_dict:
    table = table.replace(k, replace_dict[k])

print(table)

\begin{tabular}{lrrrrrrrrr}
\toprule
             name &  ship &  lego &  ficus &  drums &  mic &  materials &  hotdog &  chair &   avg \\
\midrule
  Plen (sigma=70) &    0.511 &    0.560 &  0.524 &  1.216 &   0.883 &      0.907 &      1.244 &     1.003 & 0.856 \\
Mip360 (sigma=30) &    1.122 &    0.798 &  2.163 &  2.464 &   1.470 &      1.702 &      1.482 &     1.905 & 1.638 \\
             NeuS &    0.552 &    0.619 &  1.667 & 10.273 &   0.251 &      0.210 &      0.602 &     0.210 & 1.798 \\
              HFS &    0.514 &    0.463 &  0.374 &  1.820 &   0.274 &      0.255 &      0.272 &     0.428 & 0.550 \\
   Ours (Conv Lv) &    0.336 &    0.478 &  0.281 &  0.743 &   0.313 &      0.344 &      0.814 &     0.443 & 0.469 \\
             Ours &    0.277 &    0.376 &  0.240 &  0.617 &   0.288 &      0.317 &      0.702 &     0.349 & 0.396 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_15421/2380909454.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  table = df_all.to_latex(index=False)


In [16]:
\begin{tabular}{lrrrrrrrrr}
\toprule
             name &  ship &  lego &  ficus &  drums &  mic &  materials &  hotdog &  chair &   avg \\
\midrule
  Plenoxels  &    0.511 &    0.560 &  0.524 &  1.216 &   0.883 &      0.907 &      1.244 &     1.003 & 0.856 \\
MipNeRF360  &    1.122 &    0.798 &  2.163 &  2.464 &   1.470 &      1.702 &      1.482 &     1.905 & 1.638 \\
             NeuS &    0.552 &    0.619 &  1.667 & 10.273 &   \first{0.251} &      \first{0.210} &      \second{0.602} &     \first{0.210} & 1.798 \\
              HFS &    0.514 &    \second{0.463} &  0.374 &  1.820 &   \second{0.274} &      \second{0.255} &      \first{0.272} &     0.428 & 0.550 \\
   Ours (Conv Lv) &    \second{0.336} &    0.478 &  \second{0.281} &  \second{0.743} &   0.313 &      0.344 &      0.814 &     0.443 & \second{0.469} \\
             Ours &    \first{0.277} &    \first{0.376} &  \first{0.240} &  \first{0.617} &   0.288 &      0.317 &      0.702 &     \second{0.349} & \first{0.396} \\
\bottomrule
\end{tabular}

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 5)

# Thin new

In [30]:
scene_names = [
    "ship_re", 
    "ficus", 
    "lyre",
    "bee",
    "stair",
    # "fence",
    "scale",
    "seat",
    "well",
    # "slide",
]


models = {
    # "Plen (sigma=10)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/thresh_10/ckpt/cf.txt",
    # "Plen (sigma=30)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/thresh_30/ckpt/cf.txt",
    # "Plen (sigma=50)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/thresh_50/ckpt/cf.txt",
    # "Plen (sigma=70)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/thresh_70/ckpt/cf.txt",
    "Plen (sigma=90)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/thresh_90/ckpt/cf.txt",
    # "Plen (sigma=100)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/thresh_100/ckpt/cf.txt",

    # "Mip360 (sigma=10)": "/rds/project/rds-qxpdOeYWi78/multinerf/results/blender/{}/pts/lv_10_pts/cf.txt",
    # "Mip360 (sigma=30)": "/rds/project/rds-qxpdOeYWi78/multinerf/results/blender/{}/pts/lv_30_pts/cf.txt",
    "Mip360 (sigma=50)": "/rds/project/rds-qxpdOeYWi78/multinerf/results/blender/{}/pts/lv_50_pts/cf.txt",
    # "Mip360 (sigma=70)": "/rds/project/rds-qxpdOeYWi78/multinerf/results/blender/{}/pts/lv_70_pts/cf.txt",
    # "Mip360 (sigma=90)": "/rds/project/rds-qxpdOeYWi78/multinerf/results/blender/{}/pts/lv_90_pts/cf.txt",
    # "Mip360 (sigma=100)": "/rds/project/rds-qxpdOeYWi78/multinerf/results/blender/{}/pts/lv_100_pts/cf.txt",
    # "MipNeRF360 (sigma=200)": "/rds/project/rds-qxpdOeYWi78/multinerf/results/blender/{}/pts/lv_200_pts/cf.txt",

    # "NeuS White" : "/rds/project/rds-qxpdOeYWi78/NeuS/exp/{}/womask_white/fine_mesh/cf.txt",
    "NeuS" : "/rds/project/rds-qxpdOeYWi78/NeuS/exp/{}/womask/fine_mesh/cf.txt",
    # "HFS White" : "/rds/project/rds-qxpdOeYWi78/HFS/exps_1.5/{}/womask_white_no_bg/fine_mesh/cf.txt",
    "HFS" : "/rds/project/rds-qxpdOeYWi78/HFS/exps_1.5/{}/womask_hfs/fine_mesh/cf.txt",
    "neuralangelo" : "/rds/project/rds-qxpdOeYWi78/neuralangelo/logs/syn/{}/cf.txt",
    "NeRRF" : "/rds/project/rds-qxpdOeYWi78/NeRRF/eval_log/{}/cf.txt",
    # "neuralangelo50k" : "/rds/project/rds-qxpdOeYWi78/neuralangelo/logs/syn/{}/50k/cf.txt",



    # "Ours (Conv Lv)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/good_trunc/solid_less_trunc_converge_lv/ckpt_eval_surf_masked/ckpt/cf.txt",
    "Ours": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/good_trunc/solid_less_trunc/ckpt_eval_surf_masked/ckpt/cf.txt",
    # "Ours_new_trans": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/norm_l12/norm_l12_decay_9/ckpt_eval_surf_masked/ckpt/cf.txt",
    

    # "NeRF (depth 0.1)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/depth_spiral_mode_0.1/remeshed/cf.txt",
    # # "Mip360 (depth 0.9)": "/rds/project/rds-qxpdOeYWi78/multinerf/results/blender/{}/pts/distance_mode_09_pts/cf.txt",
    # "Mip360 (depth 0.1)": "/rds/project/rds-qxpdOeYWi78/multinerf/results/blender/{}/pts/distance_mode_pts/cf.txt",

}


df_all = None

digit = 3

for i, scene in enumerate(scene_names):
    df = get_scene_comp(scene, 1e-2)
    df = df.round(digit)
    # df.loc[-1] = ["scene name", scene, scene]
    df = df.rename(columns={'Avg cf':scene})
    # df.loc[-1] = ["scene name", scene]
    # df.index = df.index + 1  # shifting index
    # df.sort_index(inplace=True)

    if i > 0:
        df = df.drop('name', axis=1)
    df_all = pd.concat([df_all, df], axis=1)

    

# df_all['mean cf'] = df_all['Avg cf'][1:].mean(axis=1).round(digit)
df_all['avg'] = df_all[scene_names].mean(axis=1).round(digit)
# df_all['mean F1'] = df_all['F1'][1:].mean(axis=1).round(digit)
df_all.style.apply(highlight_min_values)

,name,ship_re,ficus,lyre,bee,stair,scale,seat,well,avg
0,Plen (sigma=90),0.476000,0.431000,0.522000,0.541000,0.206000,0.884000,0.497000,0.647000,0.526000
1,Mip360 (sigma=50),1.217000,2.640000,1.311000,1.181000,0.279000,2.243000,0.744000,1.941000,1.445000
2,NeuS,0.552000,1.667000,0.812000,0.242000,4.087000,0.237000,0.431000,0.360000,1.049000
3,HFS,0.514000,0.374000,0.781000,0.336000,4.316000,0.271000,0.425000,0.385000,0.925000
4,neuralangelo,0.270000,0.411000,nan,0.377000,0.426000,0.789000,0.432000,0.266000,0.424000
5,NeRRF,1.740000,2.899000,nan,1.164000,3.731000,1.359000,2.170000,3.381000,2.349000
6,Ours,0.277000,0.240000,0.188000,0.207000,0.176000,0.575000,0.288000,0.319000,0.284000


In [31]:
from functools import partial

df_latex = df_all.apply(partial(mark_best_values, second=True)).to_latex()
# df_latex = df_summary.apply(partial(mark_best_values, second=True)).to_latex()
df_latex = df_latex.replace('\\textbackslash ', '\\')
df_latex = df_latex.replace('\\{', '{')
df_latex = df_latex.replace('\\}', '}')

table = df_latex

# table = df_summary.to_latex()
for k in replace_dict:
    table = table.replace(k, replace_dict[k])

print(table)

\begin{tabular}{lllllllllll}
\toprule
{} &                        name &         ship &           ficus &            lyre &             bee &           stair &           scale &            seat &            well &             avg \\
\midrule
0 &             Plen (sigma=90) &           0.476 &           0.431 &  \second{0.522} &           0.541 &  \second{0.206} &           0.884 &           0.497 &           0.647 &           0.526 \\
1 &  \second{Mip360 ($\sigma=50$)} &           1.217 &            2.64 &           1.311 &           1.181 &           0.279 &           2.243 &           0.744 &           1.941 &           1.445 \\
2 &                        NeuS &           0.552 &           1.667 &           0.812 &  \second{0.242} &           4.087 &   \first{0.237} &           0.431 &            0.36 &           1.049 \\
3 &                 \first{HFS} &           0.514 &  \second{0.374} &           0.781 &           0.336 &           4.316 &  \second{0.271} &  \second{0.425} &     

/tmp/ipykernel_47721/455206940.py:3: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  df_latex = df_all.apply(partial(mark_best_values, second=True)).to_latex()


In [ ]:
\begin{tabular}{lrrrrrrrrr}
\toprule
             name &  ship &  ficus &  lyre &   bee &  stair &  scale &  seat &  well &   avg \\
\midrule
  Plen ($\sigma=90$) &    0.476 &  0.431 & \second{0.522} & 0.541 &  \second{0.206} &  0.884 & 0.497 & 0.647 & 0.526 \\
Mip360 ($\sigma=50$) &    1.217 &  2.640 & 1.311 & 1.181 &  0.279 &  2.243 & 0.744 & 1.941 & 1.445 \\
             NeuS &    0.552 &  1.667 & 0.812 & \second{0.242} &  4.087 &  \first{0.237} & 0.431 & 0.360 & 1.049 \\
              HFS &    0.514 &  \second{0.374} & 0.781 & 0.336 &  4.316 &  \second{0.271} & \second{0.425} & 0.385 & 0.925 \\
     neuralangelo &    \first{0.270} &  0.411 &   NaN & 0.377 &  0.426 &  0.789 & 0.432 & \first{0.266} & \second{0.424} \\
             Ours &    \second{0.277} &  \first{0.240} & \first{0.188} & \first{0.207} &  \first{0.176} &  0.575 & \first{0.288} & \second{0.319} & \first{0.284} \\
\bottomrule
\end{tabular}


# Translucent

In [32]:
# scene_names = [
#     # "bulb",
#     # "crystal",
#     # "pot",
#     "glass_table",
#     "jar",
#     "glasses2",
#     "wine",
#     "lego_transparent",
#     "bottle_2",
#     "cup",
#     "case",
#     # "skull",
#     # "vase", 
#     # "perfume",
#     # "bottle",
#     # "stool",
#     # "cupboard",
# ]


models = {
    # "Plen (sigma=10)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/thresh_10/ckpt/cf.txt",
    "Plen (sigma=30)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/thresh_30/ckpt/cf.txt",
    # "Plen (sigma=50)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/thresh_50/ckpt/cf.txt",
    # "Plen (sigma=70)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/thresh_70/ckpt/cf.txt",
    # "Plen (sigma=90)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/thresh_90/ckpt/cf.txt",
    # "Plen (sigma=100)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/thresh_100/ckpt/cf.txt",

    # "Mip360 (sigma=10)": "/rds/project/rds-qxpdOeYWi78/multinerf/results/blender/{}/pts/lv_10_pts/cf.txt",
    # "Mip360 (sigma=30)": "/rds/project/rds-qxpdOeYWi78/multinerf/results/blender/{}/pts/lv_30_pts/cf.txt",
    "Mip360 (sigma=50)": "/rds/project/rds-qxpdOeYWi78/multinerf/results/blender/{}/pts/lv_50_pts/cf.txt",
    # "Mip360 (sigma=70)": "/rds/project/rds-qxpdOeYWi78/multinerf/results/blender/{}/pts/lv_70_pts/cf.txt",
    # "Mip360 (sigma=90)": "/rds/project/rds-qxpdOeYWi78/multinerf/results/blender/{}/pts/lv_90_pts/cf.txt",
    # "Mip360 (sigma=100)": "/rds/project/rds-qxpdOeYWi78/multinerf/results/blender/{}/pts/lv_100_pts/cf.txt",

    # "Plen (sigma=tau)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/thresh_10,30,50,70,90/cf.txt",


    "NeuS White" : "/rds/project/rds-qxpdOeYWi78/NeuS/exp/{}/womask_white/fine_mesh/cf.txt",
    # "NeuS" : "/rds/project/rds-qxpdOeYWi78/NeuS/exp/{}/womask/fine_mesh/cf.txt",
    "HFS White" : "/rds/project/rds-qxpdOeYWi78/HFS/exps_1.5/{}/womask_white_no_bg/fine_mesh/cf.txt",
    # "HFS" : "/rds/project/rds-qxpdOeYWi78/HFS/exps_1.5/{}/womask_hfs/fine_mesh/cf.txt",

    "neuralangelo" : "/rds/project/rds-qxpdOeYWi78/neuralangelo/logs/syn/{}/cf.txt",
    "NeRRF" : "/rds/project/rds-qxpdOeYWi78/NeRRF/eval_log/{}/cf.txt",

    # "neuralangelo50k" : "/rds/project/rds-qxpdOeYWi78/neuralangelo/logs/syn/{}/50k/cf.txt",



    # "Ours (Conv Lv)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/norm_l12/norm_l12_decay_9_low_tv_2_conv_lv/ckpt_eval_surf_masked/ckpt/cf.txt",
    "Ours": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/norm_l12/norm_l12_decay_9_low_tv_2/ckpt_eval_surf_masked/ckpt/cf.txt",

    # # "Ours": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/good_trunc/solid_less_trunc/ckpt_eval_surf_masked/ckpt/cf.txt",
    # "Ours_new_trans": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/norm_l12/norm_l12_decay_9/ckpt_eval_surf_masked/ckpt/cf.txt",
    # "Ours_new_trans conv lv": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/norm_l12/norm_l12_decay_9_converge_lv/ckpt_eval_surf_masked/ckpt/cf.txt",
    # "Ours_new_trans conv lv 2": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/norm_l12/norm_l12_decay_9_converge_lv_2/ckpt_eval_surf_masked/ckpt/cf.txt",
    # "norm_l12_decay_9_low_tv_2": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/norm_l12/norm_l12_decay_9_low_tv_2/ckpt_eval_surf_masked/ckpt/cf.txt",
    # "norm_l12_decay_9_low_tv_2_conv_lv": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/norm_l12/norm_l12_decay_9_low_tv_2_conv_lv/ckpt_eval_surf_masked/ckpt/cf.txt",
    

    # "NeRF (depth 0.1)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/depth_spiral_mode_0.1/remeshed/cf.txt",
    # "Mip360 (depth 0.9)": "/rds/project/rds-qxpdOeYWi78/multinerf/results/blender/{}/pts/distance_mode_09_pts/cf.txt",
    # "Mip360 (depth 0.1)": "/rds/project/rds-qxpdOeYWi78/multinerf/results/blender/{}/pts/distance_mode_pts/cf.txt",

}                                              

scene_names = [
    "case",
    "dinning_table",
    "coffe_table",
    "kitchen_table",
    "bottle_2",
    # "hourglass",
    "monkey",
    "double_table",
    "leaf_vase"
    # "chest",
]

# "scale" "seat" "well" "house" "swing"
df_all = None

digit = 3

for i, scene in enumerate(scene_names):
    df = get_scene_comp(scene, 1e-2)
    df = df.round(digit)
    # df.loc[-1] = ["scene name", scene, scene]
    df = df.rename(columns={'Avg cf':scene})
    # df.loc[-1] = ["scene name", scene]
    # df.index = df.index + 1  # shifting index
    # df.sort_index(inplace=True)

    if i > 0:
        df = df.drop('name', axis=1)
    df_all = pd.concat([df_all, df], axis=1)

    

# df_all['mean cf'] = df_all['Avg cf'][1:].mean(axis=1).round(digit)
df_all['avg'] = df_all[scene_names].mean(axis=1).round(digit)
# df_all['mean F1'] = df_all['F1'][1:].mean(axis=1).round(digit)
df_all

df_all.style.apply(highlight_min_values)

,name,case,dinning_table,coffe_table,kitchen_table,bottle_2,monkey,double_table,leaf_vase,avg
0,Plen (sigma=30),1.195000,0.438000,0.706000,0.378000,0.709000,1.084000,0.557000,1.024000,0.761000
1,Mip360 (sigma=50),4.012000,5.217000,2.096000,2.375000,2.324000,4.390000,2.900000,1.190000,3.063000
2,NeuS White,5.091000,1.188000,1.070000,0.392000,2.271000,5.923000,0.874000,1.946000,2.344000
3,HFS White,5.094000,0.854000,2.839000,nan,1.493000,3.223000,nan,8.684000,3.698000
4,neuralangelo,2.072000,0.483000,0.798000,0.577000,0.395000,2.464000,0.513000,1.701000,1.125000
5,NeRRF,1.267000,0.571000,0.822000,3.091000,3.720000,2.905000,0.829000,3.486000,2.086000
6,Ours,0.835000,0.373000,0.717000,0.255000,0.653000,0.776000,0.512000,0.870000,0.624000


In [33]:
from functools import partial

df_latex = df_all.apply(partial(mark_best_values, second=True)).to_latex()
# df_latex = df_summary.apply(partial(mark_best_values, second=True)).to_latex()
df_latex = df_latex.replace('\\textbackslash ', '\\')
df_latex = df_latex.replace('\\{', '{')
df_latex = df_latex.replace('\\}', '}')

table = df_latex

# table = df_summary.to_latex()
for k in replace_dict:
    table = table.replace(k, replace_dict[k])

print(table)

\begin{tabular}{lllllllllll}
\toprule
{} &                        name &            case &   dinning\_table &     coffe\_table &   kitchen\_table &        bottle\_2 &          monkey &    double\_table &       leaf\_vase &             avg \\
\midrule
0 &             Plen (sigma=30) &  \second{1.195} &  \second{0.438} &   \first{0.706} &  \second{0.378} &           0.709 &  \second{1.084} &           0.557 &  \second{1.024} &  \second{0.761} \\
1 &  \second{Mip360 ($\sigma=50$)} &           4.012 &           5.217 &           2.096 &           2.375 &           2.324 &            4.39 &             2.9 &            1.19 &           3.063 \\
2 &                  NeuS White &           5.091 &           1.188 &            1.07 &           0.392 &           2.271 &           5.923 &           0.874 &           1.946 &           2.344 \\
3 &           \first{HFS White} &           5.094 &           0.854 &           2.839 &             NaN &           1.493 &           3.223 &             N

/tmp/ipykernel_47721/455206940.py:3: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  df_latex = df_all.apply(partial(mark_best_values, second=True)).to_latex()


In [46]:
\begin{tabular}{lrrrrrrrrr}
\toprule
             name &  case &  dinning\_table &  coffe\_table &  kitchen\_table &  bottle\_2 &  monkey &  double\_table &  leaf\_vase &   avg \\
\midrule
  Plen ($\sigma=30$) & \second{1.195} &          \second{0.438} &        \first{0.706} &          \second{0.378} &     0.709 &   \second{1.084} &         0.557 &      \second{1.024} & \second{0.761} \\
Mip360 ($\sigma=50$) & 4.012 &          5.217 &        2.096 &          2.375 &     2.324 &   4.390 &         2.900 &      1.190 & 3.063 \\
       NeuS White & 5.091 &          1.188 &        1.070 &          0.392 &     2.271 &   5.923 &         0.874 &      1.946 & 2.344 \\
        HFS White & 5.094 &          0.854 &        2.839 &            NaN &     1.493 &   3.223 &           NaN &      8.684 & 3.698 \\
     neuralangelo & 2.072 &          0.483 &        0.798 &          0.577 &     \first{0.395} &   2.464 &         \second{0.513} &      1.701 & 1.125 \\
             Ours & \first{0.835} &          \first{0.373} &        \second{0.717} &          \first{0.255} &     \second{0.653} &   \first{0.776} &         \first{0.512} &      \first{0.870} & \first{0.624} \\
\bottomrule
\end{tabular}

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,scene name,NeRF (sig=10),NeRF (sig=50),NeRF (test mode 0.1),MipNeRF360 (sig=10),MipNeRF360 (sig=50),MipNeRF360 (test mode 0.9),Ours (clamp_0149_no_ldist_no_in_sparse/surf_ma...,Ours (clamp_0149_no_ldist_no_in_sparse/surf_si...,Ours (clamp_0149_mlv_2/surf_masked),Ours (clamp_0149_mlv_2/surf),NeuS 0.5,HFS 0.5
1,NaN,2.061,1.963,1.329,5.503,5.3,1.781,NaN,NaN,0.896,1.086,NaN,NaN
2,NaN,2.061,1.963,1.329,5.503,5.3,1.781,NaN,NaN,0.896,1.086,NaN,NaN


# Determine the best level set for Plenoxels and MipNeRF360

In [7]:
models = {
    "Plen (sigma=10)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/thresh_10/ckpt/cf.txt",
    "Plen (sigma=30)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/thresh_30/ckpt/cf.txt",
    "Plen (sigma=50)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/thresh_50/ckpt/cf.txt",
    "Plen (sigma=70)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/thresh_70/ckpt/cf.txt",
    "Plen (sigma=90)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/thresh_90/ckpt/cf.txt",
    "Plen (sigma=100)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/thresh_100/ckpt/cf.txt",
    # "Plen (sigma=tau)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/thresh_10,30,50,70,90/cf.txt",
    "Plen (Best)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/thresh_100/ckpt/cf.txt",
    "Plen (Best Lv)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/thresh_100/ckpt/cf.txt",


    "Mip360 (sigma=10)": "/rds/project/rds-qxpdOeYWi78/multinerf/results/blender/{}/pts/lv_10_pts/cf.txt",
    "Mip360 (sigma=30)": "/rds/project/rds-qxpdOeYWi78/multinerf/results/blender/{}/pts/lv_30_pts/cf.txt",
    "Mip360 (sigma=50)": "/rds/project/rds-qxpdOeYWi78/multinerf/results/blender/{}/pts/lv_50_pts/cf.txt",
    "Mip360 (sigma=70)": "/rds/project/rds-qxpdOeYWi78/multinerf/results/blender/{}/pts/lv_70_pts/cf.txt",
    "Mip360 (sigma=90)": "/rds/project/rds-qxpdOeYWi78/multinerf/results/blender/{}/pts/lv_90_pts/cf.txt",
    "Mip360 (sigma=100)": "/rds/project/rds-qxpdOeYWi78/multinerf/results/blender/{}/pts/lv_100_pts/cf.txt",
    "Mip360 (sigma=tau)": "/rds/project/rds-qxpdOeYWi78/multinerf/results/blender/{}/pts/cf.txt",
    "Mip360 (Best)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/thresh_100/cf.txt",
    "Mip360 (Best Lv)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/thresh_100/ckpt/cf.txt",
}


scene_names = [
    [   
    "ship_re", 
    "lego_re", 
    "ficus",
    "drums",
    "mic_re",
    "materials",
    "hotdog_re",
    "chair_re",
    ],
    [   
    "lyre",
    "bee",
    "stair",
    "fence",
    "scale",
    "seat",
    "well",
    "slide",
    ],
    [
    "case",
    "dinning_table",
    "coffe_table",
    "kitchen_table",
    "bottle_2",
    "monkey",
    "double_table",
    "leaf_vase"
    ]
]


group_names = ['NeRF Synthetic', 'Thin', 'Translucent']

digit = 3

df_summary = None

for group_i in range(len(scene_names)):

    df_group = None
    for i, scene in enumerate(scene_names[group_i]):
        df = get_scene_comp(scene, 1e-2)
        df = df.round(digit)
        # df.loc[-1] = ["scene name", scene, scene]
        df.loc[-1] = ["scene name", scene]
        df.index = df.index + 1  # shifting index
        df.sort_index(inplace=True)

        if i > 0:
            df = df.drop('name', axis=1)
        df_group = pd.concat([df_group, df], axis=1)

        

    df_group[group_names[group_i]] = df_group['Avg cf'][1:].mean(axis=1).round(digit)
    df_group = df_group[['name', group_names[group_i]]].transpose()

    if df_summary is None:
        df_summary = df_group
    else:
        df_summary = pd.concat([df_summary, df_group.iloc[[1]]], axis=0)

df_summary = df_summary.drop(columns=[0]).transpose().set_index('name')
df_summary_original = df_summary.copy()

df_summary = df_summary.rename(index={
    'Ours (solid_no_fs/surf_single)' : 'Ours',
    'Ours (solid_no_fs/surf_masked)' : 'Ours (Masked)',
    'NeRF (test mode 0.1)' : 'NeRF (depth 0.1)',
    'MipNeRF360 (test mode 0.1)' : 'MipNeRF360 (depth 0.1)',
    'MipNeRF360 (test mode 0.9)' : 'MipNeRF360 (depth 0.9)',

})


levels = [10,30,50,70,90,100]
for i in range(3):
    df_summary.at['Plen (Best)', group_names[i]] = np.min([df_summary.at[f"Plen (sigma={lv})", group_names[i]] for lv in levels])
    df_summary.at['Plen (Best Lv)', group_names[i]] = levels[np.argmin([df_summary.at[f"Plen (sigma={lv})", group_names[i]] for lv in levels])]


for i in range(3):
    df_summary.at['Mip360 (Best)', group_names[i]] = np.min([df_summary.at[f"Mip360 (sigma={lv})", group_names[i]] for lv in levels])
    df_summary.at['Mip360 (Best Lv)', group_names[i]] = levels[np.argmin([df_summary.at[f"Mip360 (sigma={lv})", group_names[i]] for lv in levels])]




df_summary
df_summary['average'] = df_summary.mean(axis=1).round(digit)
# print(df_summary.round(digit).to_latex())
df_summary.style.highlight_min(color = 'grey', axis = 0)




,NeRF Synthetic,Thin,Translucent,average
name,,,,
Plen (sigma=10),1.212000,0.595000,0.813000,0.873000
Plen (sigma=30),1.260000,0.690000,0.761000,0.904000
Plen (sigma=50),0.944000,0.609000,0.812000,0.788000
Plen (sigma=70),0.856000,0.538000,1.062000,0.819000
Plen (sigma=90),0.920000,0.522000,1.597000,1.013000
Plen (sigma=100),0.977000,0.548000,1.832000,1.119000
Plen (Best),0.856000,0.522000,0.761000,0.713000
Plen (Best Lv),70,90,30,63.333000
Mip360 (sigma=10),1.699000,1.796000,3.760000,2.418000


In [2]:
models = {
    "Plen (sigma=10)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/thresh_10/ckpt/cf.txt",
    "Plen (sigma=30)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/thresh_30/ckpt/cf.txt",
    "Plen (sigma=50)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/thresh_50/ckpt/cf.txt",
    "Plen (sigma=70)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/thresh_70/ckpt/cf.txt",
    "Plen (sigma=90)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/thresh_90/ckpt/cf.txt",
    "Plen (sigma=100)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/thresh_100/ckpt/cf.txt",
    # "Plen (sigma=tau)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/thresh_10,30,50,70,90/cf.txt",
    "Plen (Best)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/thresh_100/ckpt/cf.txt",
    "Plen (Best Lv)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/thresh_100/ckpt/cf.txt",


    "Mip360 (sigma=10)": "/rds/project/rds-qxpdOeYWi78/multinerf/results/blender/{}/pts/lv_10_pts/cf.txt",
    "Mip360 (sigma=30)": "/rds/project/rds-qxpdOeYWi78/multinerf/results/blender/{}/pts/lv_30_pts/cf.txt",
    "Mip360 (sigma=50)": "/rds/project/rds-qxpdOeYWi78/multinerf/results/blender/{}/pts/lv_50_pts/cf.txt",
    "Mip360 (sigma=70)": "/rds/project/rds-qxpdOeYWi78/multinerf/results/blender/{}/pts/lv_70_pts/cf.txt",
    "Mip360 (sigma=90)": "/rds/project/rds-qxpdOeYWi78/multinerf/results/blender/{}/pts/lv_90_pts/cf.txt",
    "Mip360 (sigma=100)": "/rds/project/rds-qxpdOeYWi78/multinerf/results/blender/{}/pts/lv_100_pts/cf.txt",
    # "Mip360 (sigma=tau)": "/rds/project/rds-qxpdOeYWi78/multinerf/results/blender/{}/pts/cf.txt",
    "Mip360 (Best)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/thresh_100/cf.txt",
    "Mip360 (Best Lv)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/thresh_100/ckpt/cf.txt",
}


scene_names = [
    [   
    "ship_re", 
    "ficus",
    "lyre",
    "bee",
    "stair",
    # "fence",
    "scale",
    "seat",
    "well",
    # "slide",
    ],
    [
    "case",
    "dinning_table",
    "coffe_table",
    "kitchen_table",
    "bottle_2",
    "monkey",
    "double_table",
    "leaf_vase"
    ]
]


group_names = ['Thin', 'Translucent']

digit = 3

df_summary = None

for group_i in range(len(scene_names)):

    df_group = None
    for i, scene in enumerate(scene_names[group_i]):
        df = get_scene_comp(scene, 1e-2)
        df = df.round(digit)
        # df.loc[-1] = ["scene name", scene, scene]
        df.loc[-1] = ["scene name", scene]
        df.index = df.index + 1  # shifting index
        df.sort_index(inplace=True)

        if i > 0:
            df = df.drop('name', axis=1)
        df_group = pd.concat([df_group, df], axis=1)

        

    df_group[group_names[group_i]] = df_group['Avg cf'][1:].mean(axis=1).round(digit)
    df_group = df_group[['name', group_names[group_i]]].transpose()

    if df_summary is None:
        df_summary = df_group
    else:
        df_summary = pd.concat([df_summary, df_group.iloc[[1]]], axis=0)

df_summary = df_summary.drop(columns=[0]).transpose().set_index('name')
df_summary_original = df_summary.copy()

df_summary = df_summary.rename(index={
    'Ours (solid_no_fs/surf_single)' : 'Ours',
    'Ours (solid_no_fs/surf_masked)' : 'Ours (Masked)',
    'NeRF (test mode 0.1)' : 'NeRF (depth 0.1)',
    'MipNeRF360 (test mode 0.1)' : 'MipNeRF360 (depth 0.1)',
    'MipNeRF360 (test mode 0.9)' : 'MipNeRF360 (depth 0.9)',

})


levels = [10,30,50,70,90,100]
for i in range(len(group_names)):
    df_summary.at['Plen (Best)', group_names[i]] = np.min([df_summary.at[f"Plen (sigma={lv})", group_names[i]] for lv in levels])
    df_summary.at['Plen (Best Lv)', group_names[i]] = levels[np.argmin([df_summary.at[f"Plen (sigma={lv})", group_names[i]] for lv in levels])]


for i in range(len(group_names)):
    df_summary.at['Mip360 (Best)', group_names[i]] = np.min([df_summary.at[f"Mip360 (sigma={lv})", group_names[i]] for lv in levels])
    df_summary.at['Mip360 (Best Lv)', group_names[i]] = levels[np.argmin([df_summary.at[f"Mip360 (sigma={lv})", group_names[i]] for lv in levels])]




df_summary
df_summary['average'] = df_summary.mean(axis=1).round(digit)
# print(df_summary.round(digit).to_latex())
df_summary.style.highlight_min(color = 'grey', axis = 0)




,Thin,Translucent,average
name,,,
Plen (sigma=10),0.759000,0.813000,0.786000
Plen (sigma=30),0.886000,0.761000,0.824000
Plen (sigma=50),0.687000,0.812000,0.750000
Plen (sigma=70),0.563000,1.062000,0.812000
Plen (sigma=90),0.526000,1.597000,1.062000
Plen (sigma=100),0.541000,1.832000,1.186000
Plen (Best),0.526000,0.761000,0.644000
Plen (Best Lv),90,30,60.000000
Mip360 (sigma=10),1.882000,3.760000,2.821000


In [3]:
replace_dict = {
    'ship\_re':'ship',
    'lego\_re':'lego',
    'mic\_re':'mic',
    'hotdog\_re':'hotdog',
    'chair\_re':'chair',
    '(sigma=10)':'($\sigma=10$)',
    '(sigma=50)':'($\sigma=50$)',
    '(sigma=100)':'($\sigma=100$)',
    '(sigma=30)':'($\sigma=30$)',
    '(sigma=70)':'($\sigma=70$)',
    '(sigma=90)':'($\sigma=90$)',
    'test mode':'depth',
    'Ours (clamp\_0149\_no\_ldist\_no\_in\_sparse/surf\_sin...':'Ours',
    'Ours (clamp\_0149\_no\_ldist\_no\_in\_sparse/surf\_mas...':'Ours (trimmed)',
    'Ours (solid\_no\_fs/surf\_single)': 'Ours',
    'Ours (solid\_no\_fs/surf\_masked)': 'Ours (trimmed)',
    'Ours (trans\_no\_fs\_low\_sparse\_2/surf\_single)': 'Ours',
    'Ours (trans\_no\_fs\_low\_sparse\_2/surf\_masked)': 'Ours (trimmed)',
}

table = df_summary.to_latex(index=True)
for k in replace_dict:
    table = table.replace(k, replace_dict[k])

print(table)

\begin{tabular}{lllr}
\toprule
{} &   Thin & Translucent &  average \\
name               &        &             &          \\
\midrule
Plen ($\sigma=10$)    &  0.759 &       0.813 &    0.786 \\
Plen ($\sigma=30$)    &  0.886 &       0.761 &    0.824 \\
Plen ($\sigma=50$)    &  0.687 &       0.812 &    0.750 \\
Plen ($\sigma=70$)    &  0.563 &       1.062 &    0.812 \\
Plen ($\sigma=90$)    &  0.526 &       1.597 &    1.062 \\
Plen ($\sigma=100$)   &  0.541 &       1.832 &    1.186 \\
Plen (Best)        &  0.526 &       0.761 &    0.644 \\
Plen (Best Lv)     &     90 &          30 &   60.000 \\
Mip360 ($\sigma=10$)  &  1.882 &        3.76 &    2.821 \\
Mip360 ($\sigma=30$)  &  1.468 &       3.081 &    2.274 \\
Mip360 ($\sigma=50$)  &  1.445 &       3.063 &    2.254 \\
Mip360 ($\sigma=70$)  &  1.526 &        3.07 &    2.298 \\
Mip360 ($\sigma=90$)  &  1.635 &       3.116 &    2.376 \\
Mip360 ($\sigma=100$) &  1.693 &       3.203 &    2.448 \\
Mip360 (Best)      &  1.445 &       3.063 & 

/tmp/ipykernel_184577/2279333426.py:22: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  table = df_summary.to_latex(index=True)


In [ ]:
\begin{tabular}{lllr}
\toprule
{} &   Thin & Translucent &  average \\
\midrule
Plen ($\sigma=10$)    &  0.759 &       0.813 &    0.786 \\
Plen ($\sigma=30$)    &  0.886 &       \first{0.761} &    0.824 \\
Plen ($\sigma=50$)    &  0.687 &       0.812 &    0.750 \\
Plen ($\sigma=70$)    &  0.563 &       1.062 &    0.812 \\
Plen ($\sigma=90$)    &  \first{0.526} &       1.597 &    1.062 \\
Plen ($\sigma=100$)   &  0.541 &       1.832 &    1.186 \\
Mip360 ($\sigma=10$)  &  1.882 &        3.76 &    2.821 \\
Mip360 ($\sigma=30$)  &  1.468 &       3.081 &    2.274 \\
Mip360 ($\sigma=50$)  &  \first{1.445} &       \first{3.063} &    2.254 \\
Mip360 ($\sigma=70$)  &  1.526 &        3.07 &    2.298 \\
Mip360 ($\sigma=90$)  &  1.635 &       3.116 &    2.376 \\
Mip360 ($\sigma=100$) &  1.693 &       3.203 &    2.448 \\
\bottomrule
\end{tabular}

# Main Table

In [28]:
models = {
    "Plen (sigma=10)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/thresh_10/ckpt/cf.txt",
    "Plen (sigma=30)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/thresh_30/ckpt/cf.txt",
    "Plen (sigma=50)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/thresh_50/ckpt/cf.txt",
    "Plen (sigma=70)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/thresh_70/ckpt/cf.txt",
    "Plen (sigma=90)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/thresh_90/ckpt/cf.txt",
    "Plen (sigma=100)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/thresh_100/ckpt/cf.txt",
    "Plen (Best)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/thresh_100/ckpt/cf.txt",

    "Mip360 (sigma=10)": "/rds/project/rds-qxpdOeYWi78/multinerf/results/blender/{}/pts/lv_10_pts/cf.txt",
    "Mip360 (sigma=30)": "/rds/project/rds-qxpdOeYWi78/multinerf/results/blender/{}/pts/lv_30_pts/cf.txt",
    "Mip360 (sigma=50)": "/rds/project/rds-qxpdOeYWi78/multinerf/results/blender/{}/pts/lv_50_pts/cf.txt",
    "Mip360 (sigma=70)": "/rds/project/rds-qxpdOeYWi78/multinerf/results/blender/{}/pts/lv_70_pts/cf.txt",
    "Mip360 (sigma=90)": "/rds/project/rds-qxpdOeYWi78/multinerf/results/blender/{}/pts/lv_90_pts/cf.txt",
    "Mip360 (sigma=100)": "/rds/project/rds-qxpdOeYWi78/multinerf/results/blender/{}/pts/lv_100_pts/cf.txt",
    "Mip360 (Best)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/thresh_100/ckpt/cf.txt",

    "NeuS White" : "/rds/project/rds-qxpdOeYWi78/NeuS/exp/{}/womask_white/fine_mesh/cf.txt",
    "NeuS" : "/rds/project/rds-qxpdOeYWi78/NeuS/exp/{}/womask/fine_mesh/cf.txt",
    "HFS White" : "/rds/project/rds-qxpdOeYWi78/HFS/exps_1.5/{}/womask_white_no_bg/fine_mesh/cf.txt",
    "HFS" : "/rds/project/rds-qxpdOeYWi78/HFS/exps_1.5/{}/womask_hfs/fine_mesh/cf.txt",
    "neuralangelo" : "/rds/project/rds-qxpdOeYWi78/neuralangelo/logs/syn/{}/cf.txt",
    # "neuralangelo50k" : "/rds/project/rds-qxpdOeYWi78/neuralangelo/logs/syn/{}/50k/cf.txt",
    "NeRRF" : "/rds/project/rds-qxpdOeYWi78/NeRRF/eval_log/{}/cf.txt",


    # "Ours": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/good_trunc/solid_less_trunc/ckpt_eval_surf_single/ckpt/cf.txt",
    "Ours": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/good_trunc/solid_less_trunc/ckpt_eval_surf_masked/ckpt/cf.txt",
    # "Ours_new_trans": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/norm_l12/norm_l12_decay_9/ckpt_eval_surf_masked/ckpt/cf.txt",
    "Ours_new_trans": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/norm_l12/norm_l12_decay_9_low_tv_2/ckpt_eval_surf_masked/ckpt/cf.txt",




    # "Ours conv": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/good_trunc/solid_less_trunc_converge_lv/ckpt_eval_surf_masked/ckpt/cf.txt",
    # "Ours conv new trans": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/norm_l12/norm_l12_decay_9_converge_lv_2/ckpt_eval_surf_masked/ckpt/cf.txt",
    # "Ours conv new trans": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/norm_l12/norm_l12_decay_9_low_tv_2_conv_lv/ckpt_eval_surf_masked/ckpt/cf.txt",


    

    # "NeRF (depth 0.1)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/nerf/syn/ckpt_eval_cuvol/depth_spiral_mode_0.1/remeshed/cf.txt",
    # "Mip360 (depth 0.9)": "/rds/project/rds-qxpdOeYWi78/multinerf/results/blender/{}/pts/distance_mode_09_pts/cf.txt",
    # "Mip360 (depth 0.1)": "/rds/project/rds-qxpdOeYWi78/multinerf/results/blender/{}/pts/distance_mode_pts/cf.txt",
    
    

    # "Ours (new_trans)": "/rds/project/rds-qxpdOeYWi78/plenoxels/opt/ckpt/tuning/{}/norm_l12/norm_l12_decay_9/ckpt_eval_surf_single/ckpt/cf.txt",

}


# scene_names = [
#     [   
#     "ship_re", 
#     "lego_re", 
#     "ficus",
#     "drums",
#     "mic_re",
#     "materials",
#     "hotdog_re",
#     "chair_re",
#     ],
#     [   
#     "lyre",
#     "bee",
#     "stair",
#     "fence",
#     "scale",
#     "seat",
#     "well",
#     "slide",
#     ],
#     [
#     "case",
#     "dinning_table",
#     "coffe_table",
#     "kitchen_table",
#     "bottle_2",
#     "monkey",
#     "double_table",
#     "leaf_vase"
#     ]
# ]
# group_names = ['NeRF Synthetic', 'Thin', 'Translucent']

scene_names = [
    [   
    "ship_re", 
    "ficus", 
    "lyre",
    "bee",
    "stair",
    "scale",
    "seat",
    "well",
    ],
    [
    "case",
    "dinning_table",
    "coffe_table",
    "kitchen_table",
    "bottle_2",
    "monkey",
    "double_table",
    "leaf_vase"
    ]
]


group_names = ['Thin', 'Translucent']

digit = 3

df_summary = None

for group_i in range(len(scene_names)):

    df_group = None
    for i, scene in enumerate(scene_names[group_i]):
        df = get_scene_comp(scene, 1e-2)
        df = df.round(digit)
        # df.loc[-1] = ["scene name", scene, scene]
        df.loc[-1] = ["scene name", scene]
        df.index = df.index + 1  # shifting index
        df.sort_index(inplace=True)

        if i > 0:
            df = df.drop('name', axis=1)
        df_group = pd.concat([df_group, df], axis=1)

        

    df_group[group_names[group_i]] = df_group['Avg cf'][1:].mean(axis=1).round(digit)
    # df_group['mean F1'] = df_group['F1'][1:].mean(axis=1).round(digit)
    df_group = df_group[['name', group_names[group_i]]].transpose()

    if df_summary is None:
        df_summary = df_group
    else:
        df_summary = pd.concat([df_summary, df_group.iloc[[1]]], axis=0)

df_summary = df_summary.drop(columns=[0]).transpose().set_index('name')
df_summary_original = df_summary.copy()
df_summary_original
# df_summary.at['Ours (clamp_0149_no_ldist_no_in_sparse/surf_masked)', 'name'] = 'Ours (Masked)'
# df_summary.at[9, 'name'] = 'Ours'

df_summary = df_summary.rename(index={
    'Ours (solid_no_fs/surf_single)' : 'Ours',
    'Ours (solid_no_fs/surf_masked)' : 'Ours (Masked)',
    'NeRF (test mode 0.1)' : 'NeRF (depth 0.1)',
    'MipNeRF360 (test mode 0.1)' : 'MipNeRF360 (depth 0.1)',
    'MipNeRF360 (test mode 0.9)' : 'MipNeRF360 (depth 0.9)',

})
# df_summary.at['Ours', group_names[2]] =  df_summary.at['Ours (trans_no_fs_low_sparse_2/surf_single)', group_names[2]]
# df_summary.at['Ours (trimmed)', group_names[2]] =  df_summary.at['Ours (trans_no_fs_low_sparse_2/surf_masked)', group_names[2]]
# df_summary.drop(['Ours (trans_no_fs_low_sparse_2/surf_single)', 'Ours (trans_no_fs_low_sparse_2/surf_masked)'], inplace=True)

df_summary.at['NeuS', group_names[-1]] =  df_summary.at['NeuS White', group_names[-1]]
df_summary.at['HFS', group_names[-1]] =  df_summary.at['HFS White', group_names[-1]]
df_summary.at['Ours', group_names[-1]] =  df_summary.at['Ours_new_trans', group_names[-1]]
# df_summary.at['Ours conv', group_names[-1]] =  df_summary.at['Ours conv new trans', group_names[-1]]
df_summary.drop(['NeuS White', 'HFS White', 'Ours_new_trans'], inplace=True)


for i in range(len(group_names)):
    df_summary.at['Plen (Best)', group_names[i]] = np.min([df_summary.at[f"Plen (sigma={lv})", group_names[i]] for lv in [10,30,50,70,90,100]])
for i in range(len(group_names)):
    df_summary.at['Mip360 (Best)', group_names[i]] = np.min([df_summary.at[f"Mip360 (sigma={lv})", group_names[i]] for lv in [10,30,50,70,90,100]])

for lv in [10,30,50,70,90,100]:
    df_summary.drop([f"Plen (sigma={lv})", f"Mip360 (sigma={lv})"], inplace=True)


df_summary
df_summary['average'] = df_summary.mean(axis=1).round(digit)
# print(df_summary.round(digit).to_latex())
df_summary.style.highlight_min(color = 'grey', axis = 0)

# df_summary = df_summary.apply(mark_best_values)

# df_summary

# df_summary.apply(mark_best_values)


,Thin,Translucent,average
name,,,
Plen (Best),0.526000,0.761000,0.644000
Mip360 (Best),1.445000,3.063000,2.254000
NeuS,1.048000,2.344000,1.696000
HFS,0.925000,3.698000,2.312000
neuralangelo,0.424000,1.125000,0.774000
NeRRF,2.349000,2.086000,2.218000
Ours,0.284000,0.624000,0.454000


In [29]:
from functools import partial

df_latex = df_summary.apply(partial(mark_best_values, second=False)).to_latex()
# df_latex = df_summary.apply(partial(mark_best_values, second=True)).to_latex()
df_latex = df_latex.replace('\\textbackslash ', '\\')
df_latex = df_latex.replace('\\{', '{')
df_latex = df_latex.replace('\\}', '}')

table = df_latex

# table = df_summary.to_latex()
for k in replace_dict:
    table = table.replace(k, replace_dict[k])

print(table)

\begin{tabular}{llll}
\toprule
{} &           Thin &    Translucent &        average \\
name          &                &                &                \\
\midrule
Plen (Best)   &          0.526 &          0.761 &          0.644 \\
Mip360 (Best) &          1.445 &          3.063 &          2.254 \\
NeuS          &          1.048 &          2.344 &          1.696 \\
HFS           &          0.925 &          3.698 &          2.312 \\
neuralangelo  &          0.424 &          1.125 &          0.774 \\
NeRRF         &          2.349 &          2.086 &          2.218 \\
Ours          &  \first{0.284} &  \first{0.624} &  \first{0.454} \\
\bottomrule
\end{tabular}



/tmp/ipykernel_47721/4094482844.py:3: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  df_latex = df_summary.apply(partial(mark_best_values, second=False)).to_latex()


In [120]:
\begin{tabular}{llllr}
\toprule
{} & Synthetic &   Thin & Translucent &  average \\
name          &                &        &             &          \\
\midrule
Plen          &          0.856 &  0.522 &       0.822 &    0.733 \\
Mip360        &          1.638 &  1.091 &        3.21 &    1.980 \\
NeuS          &          1.798 &  0.834 &       2.812 &    1.815 \\
HFS           &           0.55 &  1.086 &       2.598 &    1.411 \\
Ours          &          \first{0.396} &  \first{0.304} &       \first{0.684} &    \first{0.461} \\
\bottomrule
\end{tabular}


\begin{tabular}{lllr}
\toprule
{} &   Thin & Translucent &  average \\
name          &        &             &          \\
\midrule
Plen   &  0.526 &       0.761 &    0.644 \\
Mip360 &  1.445 &       3.063 &    2.254 \\
NeuS          &  1.048 &       2.344 &    1.696 \\
HFS           &  0.925 &       3.698 &    2.312 \\
neuralangelo  &  0.424 &       1.125 &    0.774 \\
Ours          &  \first{0.284} &       \first{0.624} &    \first{0.454} \\
\bottomrule
\end{tabular}

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 5)